In [5]:
import pandas as pd
import numpy as np

In [149]:
df = pd.read_csv('weather.csv')

def getGlobalEntropy(X):
    n = len(X)
    f_y = len(X.loc[X=='Yes'])
    if f_y == 0 or f_y ==n:
        return 0
    return -(f_y/n * np.log2(f_y/n) + (n-f_y)/n * np.log2((n-f_y)/n))

In [150]:
def get_feature_entropy(X, y):
    features = X.value_counts().to_dict()
    entropies = {}
    entropy = 0.0
    for key in features.keys():
        f_y = len(X.loc[(X==key) & (y=="Yes")])
        f_n = len(X.loc[(X==key) & (y=="No")])
        f = f_y + f_n
        if  f_y == 0:
            f_y+=f
        if f_n ==0:
            f_n +=f
        entropies[key] =-( f_y/f * np.log2(f_y/f) + f_n/f * np.log2(f_n/f))
        entropy += entropies[key]*features[key]/len(X)
    return entropy
    

In [151]:
df['Outlook'].value_counts()

Rain        5
Sunny       5
Overcast    4
Name: Outlook, dtype: int64

In [152]:
def getIG(df):
    GlobalEntropy = getGlobalEntropy(df['Decision'])
    keys = df.columns
    IG = {}
    for k in keys:
        if k == 'Decision':
            continue
        IG[k] = GlobalEntropy - get_feature_entropy(df[k],df["Decision"])
    #print(IG)
    return max(IG, key= lambda x: IG[x])

In [153]:
df.loc[(df['Outlook']=='Rain') & (df['Decision']=='Yes')]

,Day,Outlook,Temp,Humidity,Wind,Decision
3,4,Rain,70,96,Weak,Yes
4,5,Rain,68,80,Weak,Yes
9,10,Rain,75,80,Weak,Yes


In [154]:
get_feature_entropy(df['Outlook'], df['Decision'])

0.6935361388961919

In [155]:
df['Outlook'].unique()[0]

'Sunny'

In [177]:
class Tree:
    def __init__(self,name):
        self.name =name
        self.children = []
        self.col = None
        self.decision = None
        self.leaf = False
    def makeTree(self,df, method):
        if len(df['Decision'].unique()) == 1:
            self.leaf  = True
            print(df['Decision'])
            self.decison = df['Decision'][0]
            return None
        splitOnKey  = method(df)
        self.col = splitOnKey
        for u in df[splitOnKey].unique():
            self.children += [Tree(u)]
        print(len(self.children))
        for c in self.children:
            print(self.col)
            c.makeTree(df.loc[df[splitOnKey]==c.name].drop(columns=[splitOnKey]), method)
    def predict(self, inp):
        if not self.leaf:
            splitOn = inp[self.col]
            for c in self.children:
                if c.name == splitOn:
                    return c.predict(inp)
        return self.decision
            
        
        

In [178]:
t = Tree('ID3')

In [179]:
t.makeTree(df.drop(columns=["Day"]),getIG)

12
Temp
0    No
Name: Decision, dtype: object
Temp
1    No
Name: Decision, dtype: object


KeyError: 0